# 좋아할 만한 아티스트 추천 🎶

## 실습목표
- 추천시스템의 개념과 목적을 이해한다.
- Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다.
- 음악 감상 기록을 활용하여 비슷한 아티스트를 찾고 아티스트를 추천해 본다.
- 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다
- 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.
 새로운 데이터셋으로 직접 추천 모델을 만들어 본다.

## 추천시스템이란?

추천시스템은 사용자가 선호할 만한 아이템을 추측함으로써 여러 가지 항목 중 사용자에게 적합한 특정항목을 선택(information filtering)하여 제공하는 시스템

[콘텐츠 추천 알고리즘의 진화](http://www.kocca.kr/insight/vol05/vol05_04.pdf)

## 데이터 탐색하기와 전처리

In [1]:
import pandas as pd
import os

In [2]:
fname = os.getenv('HOME') + '/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 지정한 컬럼명
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


In [3]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [4]:
data['artist'] = data['artist'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [5]:
# 첫 번째 사용자가 어떤 아티스트의 노래를 듣는지 확인해봅니다.
condition = (data['user_id']== data.loc[0, 'user_id'])
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


처음 보는 아티스트가 많다.  모르면 결과를 검장 할 수 없기 때문에 추천시스템에서는 적용하는 분야에 대한 지식이 특히 더 필요한 편이다.

### 데이터 탐색

전체 데이터의 기본적인 정보를 확인한다.
- 유저수, 아티스트 수, 인기 많은 아티스트
- 유저들이 몇 명의 아티스트를 듣고 있는지에 대한 통계
- 유저 play 횟수 중앙값에 대한 통계

In [6]:
# 유저 수
print('사용자 수:',data['user_id'].nunique())

# 아티스트 수
print('아티스트 수', data['artist'].nunique())

# 인기 많은 아티스트
print('인기 많은 아티스트')
artist_count = data.groupby('artist')['user_id'].count()
print(type(artist_count))
artist_count.sort_values(ascending=False).head(30)

사용자 수: 358868
아티스트 수 291346
인기 많은 아티스트
<class 'pandas.core.series.Series'>


artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [7]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [8]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

### 모델 검증을 위한 사용자 초기 정보 세팅

> __초기 정보 세팅이 필요한 이유?__<br>
처음 접속한 사용자라면 이 사용자에 대한 아무런 초기 정보가 없는 상황이어서 사용자 기반 추천을 전혀 할 수 없어 콘텐츠 기반 필터링 방식의 추천만 가능한데, 이를 보완하여 처음부터 사용자 특성에 따른 맞춤형 서비스를 제공하기 위해.<br>

In [9]:
# 추천 시스템의 추후 검증 과정을 위해 좋아하시는 아티스트 데이터를 추가합니다.
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,black eyed peas,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30


### 모델에 활용하기 위한 전처리

데이터 관리를 위해 user와 artist에 인덱싱 작업을 해준다.

In [10]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [11]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(artist_to_idx['black eyed peas'])

358868
376


In [12]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,376,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


## 사용자의 명시적/암묵적 평가

> __Explicit Datasets__<br>
평점 기록처럼 유저가 자신의 선호도를 직접(Explicit) 표현한 Data를 Explicit Data라고 한다. 다른 종류의 Explicit Data로는 영화 리뷰, 구독, 차단 데이터 등이 있다.

> __Implicit Datasets__<br>
Implicit Data는 유저가 간접적(Implicit)으로 선호, 취향을 나타내는 데이터를 의미한다. 예시로는 검색 기록, 방문 페이지, 구매 내역 등이 있다

출처: https://orill.tistory.com/entry/Explicit-vs-Implicit-Feedback-Datasets?category=1066301

우리가 앞으로 만들어갈 모델에서는 암묵적 해석을 위해 아래의 규칙을 적용할 것이다.
1. 한 번이라도 들었다면 선호한다고 판단
2. 많이 재생한 아티스트에 대해 가중치를 주어 확실히 더 좋아한다고 판단

In [13]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')

147740,17535660
Ratio of only_one over all data is 0.84%


## Matrix Factorization(MF)

![Fast and Scalable Matrix Factorization](https://d3s0tskafalll9.cloudfront.net/media/images/E-3v2-2_ekCv9hW.max-800x600.png)
출처:https://arxiv.org/pdf/1610.05838.pdf

- (m,n) 사이즈의 행렬 R을 (m,k) 사이즈의 행렬 P와 (k,n) 사이즈의 행렬 Q로 분해한다면 R은 P와 Q의 행렬곱으로 표현 가능할 수 있다.
- 이때의 P와 Q는 Feature Matrix
- 영화를 추천하는 모델에 대입한다면, 사용자의 특성(Feature) 벡터인 P, 영화의 특성 벡터인 Q로 볼 수 있고 두 벡터를 내적해서 얻어지는 R이 사용자의 영화 선호도를 보는 것
- k는 m,n보다 작은 값이기 때문에 계산량 측면으로도 훨씬 유리하다.
- MF 모델은 성능이 준수하고 Scalability가 좋아서 많이 사용된다.

- 모델의 **목표**는 모든 유저와 아이템에 대해  **k-dimension의 벡터를 잘 만드는 것**
- 벡터를 잘 만든다는 기준은 유저i의 벡터$U_i$와 아이템j의 벡터$I_j$를 내적했을 때 유저i가 아이템j에 대해 평가한 수치$M_ij$와 비슷한가이다.
$$U_i*I_j=M_{ij}$$

이번에 사용할 모델은 [Collaborative Filtering for Implicit Feedback Datasets](http://yifanhu.net/PUB/cf.pdf)논문에서 제안한 모델을 사용할 것이다.
이 모델의 경우 아티스트를 선호하는지 안하는지를 맞추는 것이기 때문에 선호한다면 1에 가까워져야 한다.

재생횟수를 맞추는 것을 목표로 한다면 [참고 블로그](https://towardsdatascience.com/recommendation-system-matrix-factorization-d61978660b4b)의 방식으로 학습하면 된다.

## CSR(Compressed Sparse Row) Matrix

![csr matrix](https://lovit.github.io/assets/figures/sparse_matrix_csr.png)
출처:https://lovit.github.io/nlp/machine%20learning/2018/04/09/sparse_mtarix_handling/#csr-matrix

CSR Matrix는 sparse한 matrix에서 유효한 데이터의 값과 과표 정보만으로 구성되어 메모리 사용량을 최소화할 수 있는 데이터 구조이다. row 순서대로 데이터를 저장한다.

In [14]:
# csr matrix 만들기
# csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291347 sparse matrix of type '<class 'numpy.longlong'>'
	with 17535578 stored elements in Compressed Sparse Row format>

## MF 모델 학습하기

Matrix Factorization 모델을 implicit 패키지를 사용하여 학습해 본다.
- `implicit` 패키지는 implicit dataset을 사용하는 다양한 모델을 빠르게 학습할 수 있는 패키지
- `als(AlternatingLeastSquares) model`: MF에서 두 Feature Matrix를 한번에 훈련하는 것은 잘 수렴하지 않아, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 방식(AlternatingLeastSquares)이 효과적이다.

In [16]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

__AlternatingLeastSquares__ 클래스의 **\_\_init\_\_ 파라미터**를 살펴보면,

1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
3. use_gpu : GPU를 사용할 것인지 
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

1,4를 늘릴수록 학습을 잘 하게 되지만 과적합의 우려가 있다.

In [17]:
AlternatingLeastSquares.__init__

<function implicit.als.AlternatingLeastSquares.__init__(self, factors=100, regularization=0.01, dtype=<class 'numpy.float32'>, use_native=True, use_cg=True, use_gpu=False, iterations=15, calculate_training_loss=False, num_threads=0, random_state=None)>

In [18]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [19]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291347x358869 sparse matrix of type '<class 'numpy.longlong'>'
	with 17535578 stored elements in Compressed Sparse Column format>

In [20]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

모델 학습이 끝났으니 이제 아래 두 가지 사항을 살펴본다.
1. Zimin 벡터와 black eyed peas의 벡터를 어떻게 만들고 있는지
2. 두 벡터를 곱하면 어떤 값이 나오는지

In [21]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

In [22]:
zimin_vector

array([-0.1155349 ,  0.24417044, -0.44524896,  0.38379753, -0.28201425,
       -0.7313716 ,  0.08031981, -0.17844446, -0.6416532 ,  1.0218164 ,
        0.72149855, -0.1870875 ,  0.06950708,  0.3578686 ,  1.1909823 ,
        0.13683726,  0.3334549 , -0.948429  ,  0.49463892, -0.03876979,
        0.89414454, -1.0330572 ,  0.16225566, -0.40723446,  0.6624259 ,
        0.17420897,  0.9049769 , -0.5900434 , -0.63027304,  0.5529654 ,
        0.99064994, -0.09852358,  0.46570796, -0.36399487, -0.57543784,
        0.6161872 , -0.5973738 ,  0.6997717 , -0.32300997,  0.1839951 ,
        1.4268489 , -1.1376554 ,  0.7405089 , -0.4629011 ,  0.5683351 ,
        1.0577375 ,  0.544924  ,  0.18697259, -0.7108876 , -0.27253473,
       -0.10119655,  0.6795737 ,  0.02460232, -0.49889508, -1.0013518 ,
        0.02935175, -0.6140187 , -0.13328035,  0.9858695 ,  0.05322084,
        1.3278344 ,  0.1197715 , -0.04366626, -0.5681808 ,  0.08933056,
       -1.2342397 , -0.61458343,  0.26889214,  0.94548273, -0.27

In [23]:
black_eyed_peas_vector

array([ 9.50089842e-03, -4.45904536e-03,  1.26880566e-02,  1.99660677e-02,
       -3.90586065e-04, -6.33689854e-03,  7.82129914e-03,  8.95244256e-03,
        5.15646162e-03,  1.57298949e-02,  1.42898993e-03,  8.71526450e-03,
        6.19992241e-03,  1.03143975e-02,  2.12466083e-02,  1.88268609e-02,
        1.95893534e-02, -9.54828039e-03,  1.46579109e-02,  5.82491281e-03,
        2.09855232e-02, -1.64007246e-02,  1.15586249e-02,  6.31308369e-03,
        2.72102952e-02,  1.53521309e-02,  2.49136277e-02,  1.18578114e-02,
        1.11757368e-02,  3.52255180e-02,  2.02659145e-02,  3.94440955e-03,
        1.63722113e-02, -1.70520181e-03, -6.36482937e-03,  1.58640817e-02,
       -3.71999294e-03, -5.52280108e-03, -7.87207391e-03, -1.44721624e-02,
        1.71795618e-02, -2.32806467e-02,  9.89838783e-03, -4.20101872e-03,
        2.48543490e-02,  2.17212290e-02,  2.83555146e-02,  1.17464988e-02,
        1.46343058e-03,  3.34551348e-03,  4.13325895e-03,  1.87894162e-02,
        1.11471154e-02,  

In [24]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(zimin_vector, black_eyed_peas_vector)

0.49511644

In [25]:
# 모델이 나의 queen에 대한 선호도를 어떻게 예측하는지 확인한다.
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.3253079

__선호도를 예측한 결과를 어떻게 해석하면 좋을까?__

벡터 내적에 대한 기준치는 모델을 설계하는 사람이 정하기 나름이다. 객관적인 지표를 만들어 기준을 정할 수도 있고 도메인 경험을 통해 정할 수도 있다.

## 비슷한 아티스트 찾기 + 유저에게 추천하기

### 비슷한 아티스트 찾기

`AlternatingLeastSquares` 클래스에 구현되어 있는 `similar_items` 메서드를 통하여 비슷한 아티스트를 찾는다.

In [26]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist # (아티스트의 id, 유사도) Tuple 

[(62, 0.99999994),
 (277, 0.98750204),
 (5, 0.97921354),
 (28, 0.9777405),
 (217, 0.9750795),
 (473, 0.9715124),
 (247, 0.96924055),
 (418, 0.96571183),
 (490, 0.96497476),
 (694, 0.95604146),
 (1018, 0.9547756),
 (910, 0.95468545),
 (268, 0.9540946),
 (773, 0.94206744),
 (531, 0.94196063)]

In [27]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'red hot chili peppers',
 'the killers',
 'radiohead',
 'placebo',
 'the beatles',
 'u2',
 'oasis',
 'foo fighters',
 'the smashing pumpkins',
 'nirvana',
 'pink floyd',
 'nine inch nails',
 'depeche mode']

In [28]:
# 위의 코드를 함수로 생성
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

힙합 장르의 아티스트 들이 추천된다.

In [30]:
get_similar_artist('lady gaga')

['lady gaga',
 'britney spears',
 'rihanna',
 'katy perry',
 'beyoncé',
 'the pussycat dolls',
 'christina aguilera',
 'kelly clarkson',
 'mariah carey',
 'justin timberlake']

lady gaga의 경우 인기있는 pop 아티스트 들이 추천된다.

### 유저에게 아티스트 추천하기

`AlternatingLeastSquares` 클래스에 구현되어 있는 `recommend` 메서드를 통하여 제가 좋아할 만한 아티스트를 추천받는다. `filter_already_liked_items` 는 유저가 이미 평가한 아이템은 제외하는 Argument이다.

In [31]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(369, 0.48845893),
 (350, 0.46206552),
 (627, 0.4499662),
 (550, 0.44547528),
 (409, 0.42856276),
 (354, 0.42506093),
 (274, 0.4202298),
 (1800, 0.4173412),
 (564, 0.41720197),
 (2249, 0.41715783),
 (5555, 0.4058619),
 (391, 0.40345713),
 (724, 0.39985737),
 (355, 0.39907712),
 (621, 0.38814873),
 (382, 0.36976498),
 (901, 0.36822212),
 (618, 0.35468075),
 (1777, 0.34679782),
 (247, 0.34663853)]

In [32]:
[idx_to_artist[i[0]] for i in artist_recommended]

['justin timberlake',
 'rihanna',
 'maroon 5',
 'britney spears',
 'amy winehouse',
 'nelly furtado',
 'michael jackson',
 'lady gaga',
 'kanye west',
 'katy perry',
 'timbaland',
 'christina aguilera',
 'lily allen',
 'madonna',
 'alicia keys',
 'mika',
 'pink',
 'the pussycat dolls',
 'mariah carey',
 'the beatles']

filter_already_liked_items=True로 했는데 maroon 5가 결과값에 나왔다. 이것은 implicit 버전 0.4.2에서 생긴 버그이다. 

__모델은 왜 rihanna를 추천해줬을까?__<br>
`AlternatingLeastSquares` 클래스에 구현된 `explain` 메소드를 사용하면 
기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있다.

In [33]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

In [34]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]] # 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)

[('beyoncé', 0.22854058754169),
 ('black eyed peas', 0.12671617792421033),
 ('jason mraz', 0.059993191366285475),
 ('coldplay', 0.04889214648295252),
 ('maroon5', -9.06681275077744e-05)]